<a href="https://colab.research.google.com/github/saamirkc/DeepLearning/blob/main/KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
X=df.iloc[:,0:-1].values
y=df.iloc[:,-1].values

In [5]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [6]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X=sc.fit_transform(X)

In [7]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [9]:
X.shape

(768, 8)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [11]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [12]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
model.fit(X_train,y_train,epochs=100,batch_size=32)

Epoch 1/100
20/20 [==============================] - 2s 4ms/step - loss: 0.6016 - accuracy: 0.6840
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5615 - accuracy: 0.7394
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5344 - accuracy: 0.7508
Epoch 4/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5171 - accuracy: 0.7492
Epoch 5/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5041 - accuracy: 0.7590
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4949 - accuracy: 0.7671
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4881 - accuracy: 0.7655
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4829 - accuracy: 0.7671
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4791 - accuracy: 0.7687
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4759 - accuracy: 0.7704
Epoch 11/

1. how to select appropriate optimizer.
2. NO. of nodes in a layer.
3. how to select number of hidden layers.
4. all in one model



In [15]:
pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.1 MB/s eta 0:00:00


In [16]:
import kerastuner as kt

<ipython-input-16-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [17]:
def build_model(hp):
    model=Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    optimizer=hp.Choice('optimizer',['adam','rmsprop','sgd','adadelta'])
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [18]:
tuner = kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5,directory='project',project_name='diabetes')

In [19]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.6883116960525513

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 11s


In [20]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [21]:
model=tuner.get_best_models(num_models=1)[0]

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model.fit(X_train,y_train,epochs=100,batch_size=32,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 2s 16ms/step - loss: 0.5230 - accuracy: 0.7590 - val_loss: 0.4917 - val_accuracy: 0.7792
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5106 - accuracy: 0.7655 - val_loss: 0.4784 - val_accuracy: 0.7792
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5011 - accuracy: 0.7687 - val_loss: 0.4680 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4934 - accuracy: 0.7736 - val_loss: 0.4596 - val_accuracy: 0.7922
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4872 - accuracy: 0.7687 - val_loss: 0.4539 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4825 - accuracy: 0.7671 - val_loss: 0.4492 - val_accuracy: 0.7987
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4786 - accuracy: 0.7720 - val_loss: 0.4463 - val_accuracy: 0.7922
Epoch 14